In [2]:
import pandas as pd
years = [str(year) for year in range(2007, 2024)]
dataframes = []
for year in years:
    df_year = pd.read_excel('2007-2023-PIT-Counts-by-State.xlsb', sheet_name=year, engine='pyxlsb')
    df_year['Date'] = pd.to_datetime(f'{year}-01-01')
    df_year['Year'] = int(year)   
    dataframes.append(df_year)
    print(f"Processed sheet for {year}")
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_csv('combined_timeseries.csv', index=False)
print("Combined CSV with Date column saved as 'combined_timeseries.csv'")



ImportError: Missing optional dependency 'pyxlsb'.  Use pip or conda to install pyxlsb.

In [ ]:
import pandas as pd
zhvi_path = 'State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
zhvi_df = pd.read_csv(zhvi_path)
clean_columns_zhvi = {}
for col in zhvi_df.columns:
    if 'sort' in col:
        clean_columns_zhvi[col] = col.split('sort')[0]
    else:
        clean_columns_zhvi[col] = col
zhvi_df.rename(columns=clean_columns_zhvi, inplace=True)
id_vars = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName']
date_columns = [col for col in zhvi_df.columns if col not in id_vars]
melted_zhvi = pd.melt(
    zhvi_df,
    id_vars=id_vars,
    value_vars=date_columns,
    var_name='date',
    value_name='ZHVI'
)
melted_zhvi['date'] = pd.to_datetime(melted_zhvi['date'], errors='coerce')
melted_zhvi['year_month'] = melted_zhvi['date'].dt.to_period('M')
melted_zhvi.drop(columns=['StateName'], inplace=True)
macro_path = 'US_macroeconomics.csv'
macro_df = pd.read_csv(macro_path)
clean_columns_macro = {}
for col in macro_df.columns:
    if 'sort' in col:
        clean_columns_macro[col] = col.split('sort')[0]
    else:
        clean_columns_macro[col] = col
macro_df.rename(columns=clean_columns_macro, inplace=True)
macro_df['date'] = pd.to_datetime(macro_df['date'], errors='coerce')
macro_df['year_month'] = macro_df['date'].dt.to_period('M')
macro_df.drop(columns=['date'], inplace=True)
combined_df = pd.merge(
    melted_zhvi,
    macro_df,
    on='year_month',
    how='left'
)
combined_df = combined_df.sort_values(by='date').ffill()
combined_df = combined_df.sort_values(by=['RegionID', 'date']).reset_index(drop=True)
unemp_path = 'unemployment.csv'
unemp_df = pd.read_csv(unemp_path)
unemp_df.columns = [col.strip().replace(" ", "_") for col in unemp_df.columns]
unemp_df.rename(columns={"State/Area": "RegionName"}, inplace=True)
unemp_df['date'] = pd.to_datetime(
    unemp_df['Year'].astype(str) + '-' + unemp_df['Month'].astype(str) + '-01',
    errors='coerce'
)
unemp_df['year_month'] = unemp_df['date'].dt.to_period('M')
unemp_df_clean = unemp_df.drop(columns=['Year', 'Month', 'date'])
final_df = pd.merge(
    combined_df,
    unemp_df_clean,
    on=['RegionName', 'year_month'],
    how='left'
)
final_df = final_df.sort_values(by=['RegionID', 'date']).reset_index(drop=True)
final_df['Year'] = final_df['date'].dt.year
state_to_abbrev = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISIANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY"
}

final_df['RegionName'] = final_df['RegionName'].astype(str).str.strip().str.upper()
final_df['RegionName'] = final_df['RegionName'].apply(lambda x: state_to_abbrev.get(x, x))
pit_path = '/kaggle/working/combined_timeseries.csv'
pit_df = pd.read_csv(pit_path, parse_dates=['Date'])
pit_df.rename(columns={'State': 'RegionName'}, inplace=True)
pit_df['RegionName'] = pit_df['RegionName'].astype(str).str.strip().str.upper()
pit_df['Year'] = pit_df['Date'].dt.year
merged_df = pd.merge(
    final_df,
    pit_df,
    on=['RegionName', 'Year'],
    how='left',
    suffixes=("", "_PIT")
)
merged_df = merged_df.sort_values(by=['RegionID', 'date']).reset_index(drop=True)
output_path = 'zvi_macro_unemployment_PIT.csv'
merged_df.to_csv(output_path, index=False)
print(f"Final merged CSV saved as '{output_path}'")


